In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
import os,sys,inspect
import os
import joblib
import tensorflow as tf
import numpy as np
import h5py
import scipy.sparse.linalg as la
import scipy.sparse as sp
import scipy
import time
import pickle

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

import scipy.io as sio
import process_data

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-21 09:57:13--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.54.84.112, 52.201.75.180, 34.206.253.53, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.54.84.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.28M  8.40MB/s    in 1.7s    

2019-04-21 09:57:15 (8.40 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [6]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')


! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"



class GCN:
    
    """
    The neural network model.
    """
    
    def frobenius_norm(self, tensor):
        square_tensor = tf.square(tensor)
        tensor_sum = tf.reduce_sum(square_tensor)
        frobenius_norm = tf.sqrt(tensor_sum)
        return frobenius_norm
    
    def convert_coo_to_sparse_tensor(self, L):
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data.astype('float32'), L.shape)
        L = tf.sparse_reorder(L)
        return L
      
    def concat(self,x,y):
      if x is None:
        return y
      if y is None:
        return x
      return tf.concat([x,y],axis=1)
    
    def get_variable(self,name,shape,reg=False):
        v = tf.get_variable(name,shape=shape,initializer=tf.contrib.layers.xavier_initializer())
        if reg:
          self.reg_variables.append(v)
        return v
      
    def get_regularized_variable(self,name,shape):
      return self.get_variable(name,shape,True)
    
    def get_bias(self,name,reg=False):
      v = tf.get_variable(name,[1], tf.float32,initializer=tf.constant_initializer(0.1))
      if reg:
          self.reg_variables.append(v)
      return v
    
    def get_regularized_bias(self,name):
      return self.get_bias(name,True)
    
    def __init__(self, A, X, Y,
                 num_hidden_feat,
                 K,
                 learning_rate,
                 gamma, # l2 regularization parameter
                 idx_gpu = '/gpu:2'):
        
        self.num_hidden_feat = num_hidden_feat
        self.learning_rate = learning_rate
        self.gamma=gamma
        self.num_layers=len(num_hidden_feat)
        
        self.K = K
        self.reg_variables = []
        with tf.Graph().as_default() as g:
                self.graph = g
                
                with tf.device(idx_gpu):
                        # dimensions
                        M = A.shape[0]
                  
                        #definition of constant matrices
                        self.A = tf.sparse.to_dense(self.convert_coo_to_sparse_tensor(A.tocoo()))
                        self.X = tf.constant(X, dtype=tf.float32) 
                        self.Y = tf.constant(Y, dtype=tf.float32)
                        
                        #placeholder definition
                        self.idx_nodes = tf.placeholder(tf.int32)
                        self.keep_prob = tf.placeholder(tf.float32)
                        
                        #model definition
                        
                        M = self.A.get_shape().as_list()[0]
                        A_ = tf.add(self.A,tf.eye(M,M))
                        
                        H_prev = self.X
                        
                        for layer in range(self.num_layers):
                          
                          with tf.name_scope('layer_{0}'.format(layer)):

                            Fout = self.num_hidden_feat[layer]
                            Fin = H_prev.get_shape().as_list()[1]

                            H_prev = tf.nn.dropout(H_prev, self.keep_prob)
                            H = None

                            for k in range(self.K[layer]):

                              W = self.get_regularized_variable("W_{0}_{1}".format(k,layer),shape=[Fin,Fout])
                              A_1 = self.get_regularized_variable("A_1_{0}_{1}".format(k,layer),shape=[Fout,1])
                              A_2 = self.get_regularized_variable("A_2_{0}_{1}".format(k,layer),shape=[Fout,1])
                              b_1 = self.get_regularized_bias("b_1_{0}_{1}".format(k,layer))
                              b_2 = self.get_regularized_bias("b_2_{0}_{1}".format(k,layer))

                              U = tf.matmul(H_prev,W) # M x Fout

                              gamma_1 = tf.matmul(U,A_1) + b_1 # M x 1
                              gamma_2 = tf.matmul(U,A_2) + b_2 # M x 1

                              phi = tf.add(
                                  tf.multiply(A_,gamma_1),
                                  tf.multiply(A_,tf.transpose(gamma_2))) # M x M

                              phi = tf.nn.leaky_relu(phi,alpha=0.2)
                              
                              psi = tf.exp(phi)
                              psi = tf.multiply(psi,A_)
                              
                              theta = tf.reduce_sum(psi,axis=-1)
                              
                              Q = tf.multiply(psi,tf.reciprocal(theta))
                           
                              Q = tf.nn.dropout(Q, self.keep_prob)
                              H_temp = tf.matmul(Q,U) # M x Fout                          
                              H = self.concat(H,H_temp)

                            H_prev = H

                            if layer != self.num_layers-1 : # it is not the last layer
                              H_prev = tf.nn.elu(H_prev)
                        
                        self.logits = H_prev
                        
                        self.l_out = tf.gather(self.logits, self.idx_nodes)
                        self.c_Y = tf.gather(self.Y, self.idx_nodes)
                        
                        #loss function definition
                        
                        with tf.name_scope('loss'):
                          self.l2_reg = 0
                          for W in self.reg_variables:
                            self.l2_reg += tf.nn.l2_loss(W)

                          self.data_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.l_out, labels=self.c_Y)) 
                          self.loss = self.data_loss + self.gamma*self.l2_reg
                        
                        #solver definition
                        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
                        self.opt_step = self.optimizer.minimize(self.loss)
                        
                        #predictions and accuracy extraction
                        self.c_predictions = tf.argmax(tf.nn.softmax(self.l_out), 1)
                        self.accuracy = tf.contrib.metrics.accuracy(self.c_predictions, tf.argmax(self.c_Y, 1))
                        
                        #gradients computation
                        self.trainable_variables = tf.trainable_variables()
                        self.var_grad = tf.gradients(self.loss, tf.trainable_variables())
                        
                        #for i,t in enumerate(tf.trainable_variables()):
                        #  print(str(t) + " " + str(self.var_grad[i]))
                        
                        self.norm_grad = self.frobenius_norm(tf.concat([tf.reshape(g, [-1]) for g in self.var_grad], 0))
                        
                        #session creation
                        config = tf.ConfigProto(allow_soft_placement = True)
                        config.gpu_options.allow_growth = True
                        self.session = tf.Session(config=config)
                        
                        writer = tf.summary.FileWriter(LOG_DIR, self.session.graph)


                        #session initialization
                        init = tf.global_variables_initializer()
                        self.session.run(init)
                        
#learning parameters and path dataset

num_total_iter_training = 3000
learning_rate = 0.005
val_test_interval = 1
num_hidden_feat = [8,7]
K = [8,1]
gamma = 0.0005
path_dataset = './CORA/dataset.pickle'
    
#dataset loadina

A, X, Y, train_idx, val_idx, test_idx = process_data.load_data("cora")
X = process_data.preprocess_features(X)

# compute GCN adj matrix
A_tilde = sp.csr_matrix(A)
A_tilde.setdiag(1)
D = A_tilde.sum(axis=1)

D_rows, D_cols = D.nonzero()
D_vals = [D[i,j] for i, j in zip(D_rows, D_cols)]
D_vals = np.reciprocal(np.sqrt(np.asarray(D_vals)))

D_inv_sqrt = sp.csr_matrix((D_vals, (range(len(D_vals)), range(len(D_vals)))))

A_tilde = D_inv_sqrt.dot(A_tilde).dot(D_inv_sqrt)
A_tilde = A_tilde.tocsr()
A_tilde.eliminate_zeros()

# Training

num_exp = 10 #number of times training GCN over the given dataset

list_all_acc = []
list_all_cost_val_avg  = []
list_all_data_cost_val_avg = []
list_all_acc_val_avg   = []
list_all_cost_test_avg = []
list_all_acc_test_avg  = []

num_done = 0
for seed in range(num_exp):
    GCNN = GCN(A=A_tilde, X=X, Y=Y, num_hidden_feat=num_hidden_feat, K=K, learning_rate=learning_rate, gamma=gamma)

    cost_train_avg      = []
    grad_norm_train_avg = []
    acc_train_avg       = []
    cost_test_avg       = []
    grad_norm_test_avg  = []
    acc_test_avg        = []
    cost_val_avg        = []
    data_cost_val_avg   = []
    acc_val_avg         = []
    iter_test           = []
    list_training_time = list()

    #Training code
    for i in range(num_total_iter_training):
        if (len(cost_train_avg) % val_test_interval) == 0:

            #Validate the model
            tic = time.time()
            
            feed_dict = {GCNN.idx_nodes: val_idx, GCNN.keep_prob:1.0}
            acc_val, cost_val, data_cost_val = GCNN.session.run([GCNN.accuracy, GCNN.loss, GCNN.data_loss], feed_dict)
            
            data_cost_val_avg.append(data_cost_val)
            cost_val_avg.append(cost_val)
            acc_val_avg.append(acc_val)

            #Test the model
            tic = time.time()
            
            feed_dict = {GCNN.idx_nodes: test_idx, GCNN.keep_prob:1.0}
            acc_test, cost_test = GCNN.session.run([GCNN.accuracy, GCNN.loss], feed_dict)
            
            cost_test_avg.append(cost_test)
            acc_test_avg.append(acc_test)
            iter_test.append(len(cost_train_avg))

        tic = time.time()

        tic = time.time()
        feed_dict = {GCNN.idx_nodes: train_idx, GCNN.keep_prob: 0.5}
        
        _, current_training_loss, norm_grad, current_acc_training = GCNN.session.run([GCNN.opt_step, GCNN.loss, GCNN.norm_grad, GCNN.accuracy], feed_dict) 

        training_time = time.time() - tic   

        cost_train_avg.append(current_training_loss)
        grad_norm_train_avg.append(norm_grad)
        acc_train_avg.append(current_acc_training)

    #Compute and print statistics of the last realized experiment
    list_all_acc.append(100*(np.asarray(acc_test_avg)[np.asarray(data_cost_val_avg)==np.min(data_cost_val_avg)]))
    list_all_cost_val_avg.append(cost_val_avg)
    list_all_data_cost_val_avg.append(data_cost_val_avg)
    list_all_acc_val_avg.append(acc_val_avg)
    list_all_cost_test_avg.append(cost_test_avg)
    list_all_acc_test_avg.append(acc_test_avg)

    print('Num done: %d' % num_done)
    print('Max accuracy on test set achieved: %f%%' % np.max(np.asarray(acc_test_avg)*100))
    print('Max suggested accuracy: %f%%' % (100*(np.asarray(acc_test_avg)[np.asarray(data_cost_val_avg)==np.min(data_cost_val_avg)]),))
    print('Current mean: %f%%' % np.mean(list_all_acc))
    print('Current std: %f' % np.std(list_all_acc))

    num_done += 1

http://01c738b7.ngrok.io
(2708, 2708)
(2708, 1433)


/usr/local/lib/python3.6/dist-packages/scipy/sparse/compressed.py:708: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self[i, j] = values


Num done: 0
Max accuracy on test set achieved: 79.799995%
Max suggested accuracy: 79.500000%
Current mean: 79.500000%
Current std: 0.000000
Num done: 1
Max accuracy on test set achieved: 80.400002%
Max suggested accuracy: 79.699997%
Current mean: 79.599998%
Current std: 0.099998
Num done: 2
Max accuracy on test set achieved: 80.299995%
Max suggested accuracy: 79.699997%
Current mean: 79.633331%
Current std: 0.094279
Num done: 3
Max accuracy on test set achieved: 80.699997%
Max suggested accuracy: 80.400002%
Current mean: 79.824997%
Current std: 0.341871
Num done: 4
Max accuracy on test set achieved: 81.099998%
Max suggested accuracy: 80.900002%
Current mean: 80.039993%
Current std: 0.527638
Num done: 5
Max accuracy on test set achieved: 80.699997%
Max suggested accuracy: 80.299995%
Current mean: 80.083328%
Current std: 0.491314
Num done: 6
Max accuracy on test set achieved: 80.400002%
Max suggested accuracy: 79.299995%
Current mean: 79.971428%
Current std: 0.531077
Num done: 7
Max accu

In [7]:
#Print average performance
print(np.mean(list_all_acc))
print(np.std(list_all_acc))

79.85
0.48631346
